In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder , LabelEncoder
from sklearn.decomposition import PCA
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import time
from sklearn.svm import SVC



In [1865]:
#딥러닝 모델 전처리 새로운 파생변수에 그냥 trati 원핫 추가

In [1866]:
info = pd.read_csv('snp_info.csv')  #chrom :염색체 번호 cm : snp의 유전적 위치 pos :snp의 물리적 위치
info

,SNP_id,name,chrom,cm,pos
0,SNP_01,BTA-19852-no-rs,2,67.05460,42986890
1,SNP_02,ARS-USMARC-Parent-DQ647190-rs29013632,6,31.15670,13897068
2,SNP_03,ARS-BFGL-NGS-117009,6,68.28920,44649549
3,SNP_04,ARS-BFGL-NGS-60567,6,77.87490,53826064
4,SNP_05,BovineHD0600017032,6,80.50150,61779512
5,SNP_06,BovineHD0600017424,6,80.59540,63048481
6,SNP_07,Hapmap49442-BTA-111073,6,80.78000,64037334
7,SNP_08,BovineHD0600018638,6,82.68560,67510588
8,SNP_09,ARS-BFGL-NGS-37727,6,86.87400,73092782
9,SNP_10,BTB-01558306,7,62.06920,40827112


In [1867]:
train_df = pd.read_csv('train.csv')  #염색체 별로 그룹화 ,  #각 유전자 정보마다의 위치가 필요할까..?
                                     # 그룹화 한 그룹마다 두개씩 묶어서 파생변수 만들기. 6번 9번 염색체
train_df

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,...,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,...,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,...,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,TRAIN_003,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,TRAIN_004,0,0,0,2,G G,G G,C C,A A,C C,...,A A,A A,A A,G G,A A,A A,A G,A A,G A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,TRAIN_257,0,0,0,2,A G,A G,A A,G A,C C,...,A A,G A,A A,G G,A G,G A,A A,A A,A A,B
258,TRAIN_258,0,0,0,2,G G,A A,C A,A A,A A,...,G A,G A,A A,A G,A G,A A,A G,A A,G A,C
259,TRAIN_259,0,0,0,1,A G,G G,A A,G A,A A,...,G G,G A,G A,A A,G G,G G,G G,C A,G G,A
260,TRAIN_260,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G A,A G,A G,G A,G G,C A,G G,A


###EDA

In [1869]:
import shap

In [1882]:
train_x.corr()

,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,...,SNP_08A,SNP_09A,SNP_09G,SNP_10A,SNP_11A,SNP_11G,SNP_13A,SNP_14C,SNP_15A,SNP_15G
SNP_01,1.000000,-0.367094,0.471602,-0.384073,0.449491,-0.432030,-0.627443,0.460054,-0.460103,0.508486,...,-0.460054,0.460103,-0.460103,-0.508486,0.387304,-0.387304,0.391330,-0.554781,0.449802,-0.449802
SNP_02,-0.367094,1.000000,-0.260399,0.405666,-0.119215,0.357868,0.478507,-0.295916,0.410250,-0.262874,...,0.295916,-0.410250,0.410250,0.262874,-0.392662,0.392662,-0.167716,0.344393,-0.346374,0.346374
SNP_03,0.471602,-0.260399,1.000000,-0.341657,0.282002,-0.347028,-0.518701,0.526315,-0.417378,0.472261,...,-0.526315,0.417378,-0.417378,-0.472261,0.311461,-0.311461,0.273851,-0.471268,0.297153,-0.297153
SNP_04,-0.384073,0.405666,-0.341657,1.000000,-0.260659,0.456988,0.555693,-0.286681,0.513975,-0.298976,...,0.286681,-0.513975,0.513975,0.298976,-0.399567,0.399567,-0.265730,0.433585,-0.291331,0.291331
SNP_05,0.449491,-0.119215,0.282002,-0.260659,1.000000,-0.340798,-0.598221,0.308640,-0.346092,0.456613,...,-0.308640,0.346092,-0.346092,-0.456613,0.169929,-0.169929,0.527582,-0.417456,0.357879,-0.357879
SNP_06,-0.432030,0.357868,-0.347028,0.456988,-0.340798,1.000000,0.472210,-0.398591,0.490965,-0.391743,...,0.398591,-0.490965,0.490965,0.391743,-0.382152,0.382152,-0.349076,0.360762,-0.355482,0.355482
SNP_07,-0.627443,0.478507,-0.518701,0.555693,-0.598221,0.472210,1.000000,-0.490662,0.637505,-0.570065,...,0.490662,-0.637505,0.637505,0.570065,-0.529926,0.529926,-0.496966,0.580901,-0.552935,0.552935
SNP_08,0.460054,-0.295916,0.526315,-0.286681,0.308640,-0.398591,-0.490662,1.000000,-0.485897,0.580006,...,-1.000000,0.485897,-0.485897,-0.580006,0.247170,-0.247170,0.416488,-0.553108,0.375803,-0.375803
SNP_09,-0.460103,0.410250,-0.417378,0.513975,-0.346092,0.490965,0.637505,-0.485897,1.000000,-0.442063,...,0.485897,-1.000000,1.000000,0.442063,-0.427825,0.427825,-0.354706,0.442092,-0.367630,0.367630
SNP_10,0.508486,-0.262874,0.472261,-0.298976,0.456613,-0.391743,-0.570065,0.580006,-0.442063,1.000000,...,-0.580006,0.442063,-0.442063,-1.000000,0.342805,-0.342805,0.451141,-0.568852,0.475274,-0.475274


In [ ]:
train_df =train_df.iloc[:,4:]

In [ ]:
lbe = LabelEncoder()
for i in list(train_df.columns):
    train_df[i] = lbe.fit_transform(train_df[i])


In [ ]:
train_x.corr()[train_x.corr()>0.9].info()# 다중공선성 x
#다 사용해야할듯

In [ ]:
train_x

,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,...,SNP_10A,SNP_10G,SNP_11A,SNP_11G,SNP_13A,SNP_13G,SNP_14A,SNP_14G,SNP_15A,SNP_15G
0,2,1,0,1,1,0,0,2,0,2,...,0,2,1,1,2,0,2,0,2,0
1,1,1,1,0,0,1,0,1,0,1,...,1,1,2,0,0,2,2,0,2,0
2,2,2,0,1,2,2,0,1,1,1,...,1,1,2,0,2,0,2,0,2,0
3,0,2,0,1,0,2,2,0,2,1,...,1,1,0,2,0,2,2,0,0,2
4,2,2,2,0,2,0,0,0,0,2,...,0,2,2,0,1,1,2,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,1,1,0,1,2,1,0,1,0,2,...,0,2,1,1,2,0,2,0,2,0
258,2,0,1,0,0,1,1,1,0,1,...,1,1,1,1,1,1,2,0,1,1
259,1,2,0,1,0,1,2,1,1,0,...,2,0,0,2,0,2,1,0,0,2
260,0,2,0,1,0,2,2,0,1,1,...,1,1,1,1,0,2,1,0,0,2


In [ ]:
!pip install eli5

In [3928]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(lgbm_rgs_model, scoring = "f1_macro", random_state = 42).fit(train_x, train_y)
eli5.show_weights(perm, top = 80, feature_names = train_x.columns.tolist())

Weight,Feature
0.2215 ± 0.0374,trait
0.1027 ± 0.0189,SNP_05
0.0826 ± 0.0287,SNP_04
0.0700 ± 0.0195,SNP_10
0.0638 ± 0.0204,SNP_08
0.0231 ± 0.0100,SNP_11
0.0216 ± 0.0102,SNP_13
0.0170 ± 0.0145,SNP_01
0.0141 ± 0.0045,SNP_15


In [ ]:
train_x.drop(columns=['SNP_07A','SNP_14'],inplace=True)
test_df.drop(columns=['SNP_07A','SNP_14'],inplace=True)

In [2928]:
train_df['SNP_15'][train_df['class']=='B'].value_counts()

A A    71
G A    40
G G     3
Name: SNP_15, dtype: int64

In [2929]:
train_df['SNP_15'][train_df['class']=='C'].value_counts()

G A    39
A A    32
G G     8
Name: SNP_15, dtype: int64

In [2231]:
# 위에서 뭘 더 어떻게 할까...

전처리

In [ ]:
# ridge
# only라벨인코딩 :0.9574093507426841 
# only원핫인코딩 : 0.96
# trait 원핫 나머지 라벨인코딩 : 0.9647027980361313
# lgbm
# only라벨인코딩 :0.9536078736078736
# only원핫인코딩 : 0.949
# trait 원핫 나머지 라벨인코딩 :0.9501231041492716
# SNP02 SNP06 trait 원핫 나머지 라벨:0.954011876665085

In [7]:
test_df = pd.read_csv('test.csv')

In [8]:
train_df = pd.read_csv('train.csv')

     ---------------------------------------- 1.1/1.1 MB 11.6 MB/s eta 0:00:00
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.2.0
    Uninstalling Pygments-2.2.0:
      Successfully uninstalled Pygments-2.2.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
metrics 0.3.3 requires Pygments==2.2.0, but you have pygments 2.13.0 which is incompatible.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.


In [ ]:
train_x = train_df.iloc[:,4:-1]
test_df = test_df.iloc[:,4:]


In [4255]:
train_y = train_df.iloc[:,-1]

In [ ]:
lbe = LabelEncoder()
for i in train_x.columns:
    lbe.fit(train_x[i])
    train_x[i] = lbe.transform(train_x[i])
    test_df[i] = lbe.transform(test_df[i])



In [4257]:
train_y = pd.DataFrame(train_y)

In [4235]:

lbe_y = LabelEncoder()
train_y['class'] = train_y['class'].astype('category')
train_y['class'] =lbe_y.fit_transform(train_y['class'])


In [4160]:
train_y['class'] = train_y['class'].astype('category')
train_y = pd.get_dummies(train_y)

In [3107]:
train_x.columns

Index(['trait', 'SNP_01', 'SNP_02', 'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06',
       'SNP_07', 'SNP_08', 'SNP_09', 'SNP_10', 'SNP_11', 'SNP_12', 'SNP_13',
       'SNP_14', 'SNP_15'],
      dtype='object')

In [4258]:
train_x = train_x.drop(columns=['SNP_02','SNP_03','SNP_06','SNP_09','SNP_11','SNP_12'])
test_df = test_df.drop(columns=['SNP_02','SNP_03','SNP_06','SNP_09','SNP_11','SNP_12'])

In [ ]:
# for i in list(train_x.columns): #아그맥스 사용하기
#      ohe=OneHotEncoder(sparse=False)
#      ohe.fit(np.array(train_x[i]).reshape(-1,1))
#      train_x[i]=ohe.transform(np.array(train_x[i]).reshape(-1,1))
#      test_df[i]=ohe.transform(np.array(test_df[i]).reshape(-1,1))

In [4199]:
# train_x['trait'] = train_x['trait'].astype('category')
# test_df['trait'] = test_df['trait'].astype('category')

train_x['SNP_01'] = train_x['SNP_01'].astype('category')
test_df['SNP_01'] = test_df['SNP_01'].astype('category')
# train_x['SNP_03'] = train_x['SNP_03'].astype('category')
# test_df['SNP_03'] = test_df['SNP_03'].astype('category')
train_x['SNP_04'] = train_x['SNP_04'].astype('category')
test_df['SNP_04'] = test_df['SNP_04'].astype('category')
train_x['SNP_05'] = train_x['SNP_05'].astype('category')
test_df['SNP_05'] = test_df['SNP_05'].astype('category')
# train_x['SNP_06'] = train_x['SNP_06'].astype('category')
# test_df['SNP_06'] = test_df['SNP_06'].astype('category')
# train_x['SNP_07'] = train_x['SNP_07'].astype('category')
# test_df['SNP_07'] = test_df['SNP_07'].astype('category')
# train_x['SNP_02'] = train_x['SNP_02'].astype('category')
# test_df['SNP_02'] = test_df['SNP_02'].astype('category')
train_x['SNP_08'] = train_x['SNP_08'].astype('category')
test_df['SNP_08'] = test_df['SNP_08'].astype('category')
# train_x['SNP_09'] = train_x['SNP_09'].astype('category')
# test_df['SNP_09'] = test_df['SNP_09'].astype('category')
train_x['SNP_10'] = train_x['SNP_10'].astype('category')
test_df['SNP_10'] = test_df['SNP_10'].astype('category')
# train_x['SNP_11'] = train_x['SNP_11'].astype('category')
# test_df['SNP_11'] = test_df['SNP_11'].astype('category')
# train_x['SNP_12'] = train_x['SNP_12'].astype('category')
# test_df['SNP_12'] = test_df['SNP_12'].astype('category')
train_x['SNP_13'] = train_x['SNP_13'].astype('category')
test_df['SNP_13'] = test_df['SNP_13'].astype('category')
# train_x['SNP_14'] = train_x['SNP_14'].astype('category')
# test_df['SNP_14'] = test_df['SNP_14'].astype('category')
train_x['SNP_15'] = train_x['SNP_15'].astype('category')
test_df['SNP_15'] = test_df['SNP_15'].astype('category')
train_x = pd.get_dummies(train_x)
test_df = pd.get_dummies(test_df)

In [2938]:
train_x.columns

Index(['trait', 'SNP_01_0', 'SNP_01_1', 'SNP_01_2', 'SNP_02_0', 'SNP_02_1',
       'SNP_02_2', 'SNP_03_0', 'SNP_03_1', 'SNP_03_2', 'SNP_04_0', 'SNP_04_1',
       'SNP_04_2', 'SNP_05_0', 'SNP_05_1', 'SNP_05_2', 'SNP_06_0', 'SNP_06_1',
       'SNP_06_2', 'SNP_07_0', 'SNP_07_1', 'SNP_07_2', 'SNP_08_0', 'SNP_08_1',
       'SNP_08_2', 'SNP_09_0', 'SNP_09_1', 'SNP_09_2', 'SNP_10_0', 'SNP_10_1',
       'SNP_10_2', 'SNP_11_0', 'SNP_11_1', 'SNP_11_2', 'SNP_12_0', 'SNP_12_1',
       'SNP_12_2', 'SNP_13_0', 'SNP_13_1', 'SNP_13_2', 'SNP_14_0', 'SNP_14_1',
       'SNP_14_2', 'SNP_15_0', 'SNP_15_1', 'SNP_15_2'],
      dtype='object')

In [3079]:
train_x.drop(columns=['SNP_02_0','SNP_02_1', 'SNP_02_2', 'SNP_03_0','SNP_03_1', 'SNP_03_2','SNP_06_0',
       'SNP_06_1','SNP_06_2','SNP_11_0', 'SNP_11_1','SNP_11_2','SNP_12_0', 'SNP_12_1',
       'SNP_12_2'],inplace=True)
test_df.drop(columns=['SNP_02_0','SNP_02_1', 'SNP_02_2', 'SNP_03_0','SNP_03_1', 'SNP_03_2','SNP_06_0',
       'SNP_06_1','SNP_06_2','SNP_11_0', 'SNP_11_1','SNP_11_2','SNP_12_0', 'SNP_12_1',
       'SNP_12_2'],inplace=True)

In [2367]:
x = pd.read_csv('train 극한삭제.csv')
y = pd.read_csv('test 극한삭제.csv')

In [2368]:
x.drop(columns=['Unnamed: 0'],inplace=True)
y.drop(columns=['Unnamed: 0'],inplace=True)

In [2369]:
x1 = train_x.iloc[:,0]
y1 = test_df.iloc[:,0]

In [2370]:
x1 = pd.DataFrame(x1)
y1 = pd.DataFrame(y1)

In [2371]:
train_x = pd.concat([x1,x],axis=1)
test_df = pd.concat([y1,y],axis=1)

In [ ]:
train_x = pd.concat([train_x,x],axis=1)
test_df = pd.concat([test_df,y],axis=1)

모델링

In [4131]:
from sklearn.linear_model import RidgeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import ElasticNetCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,learning_curve, cross_val_score, KFold
import optuna
from sklearn.metrics import accuracy_score ,f1_score
from optuna.integration import XGBoostPruningCallback
import lightgbm as lgb
from optuna.samplers import TPESampler


In [4259]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty = 'elasticnet',solver="saga")
t1 = time.time()
lr_parameters={
    'class_weight':['dict','balanced'],
    'C':[1,2,3,4,5],
    'l1_ratio' :[0.3,0.5,0.7,0.8,0.9],
    'max_iter':[90,100,110,120],
    'tol':[0.001,0.0001,0.01,0.1,0.2,0.00001]
    }
    
# RS_iter
n_iter_search =20

# RandomSearchCV
lr_rgs_model = RandomizedSearchCV(
    lr, 
    param_distributions=lr_parameters,
    n_iter = n_iter_search,
    cv=3,
    random_state=42,
    n_jobs=-1,
    scoring="f1_macro"
)
lr_rgs_model.fit(train_x, train_y)
t2 = time.time()
print("Processed Time:", str(t2-t1))

Processed Time: 0.12309789657592773


In [4260]:
lr_rgs_model.best_params_

{'tol': 0.01,
 'max_iter': 90,
 'l1_ratio': 0.7,
 'class_weight': 'balanced',
 'C': 1}

In [4261]:
lr_rgs_model.best_score_

0.964753473529031

In [4262]:
lr_rgs_best = lr_rgs_model.best_estimator_
pred = lr_rgs_model.predict(test_df)
pred

array(['A', 'B', 'C', 'B', 'A', 'B', 'C', 'B', 'A', 'A', 'C', 'B', 'C',
       'A', 'B', 'B', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'B',
       'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'B', 'B',
       'C', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B',
       'A', 'B', 'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C',
       'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'B',
       'B', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'A', 'B',
       'B', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'C',
       'A', 'A', 'C', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A',
       'C', 'B', 'C', 'A', 'B', 'B', 'C', 'C', 'B', 'C', 'B', 'A', 'B',
       'A', 'A', 'B', 'B', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'A', 'C',
       'B', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A',
       'A', 'B', 'C', 'A', 'B', 'C', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'C', 'C', 'B', 'B'], dtype=object)

In [4246]:
def F1(x, y_pred):
    f1score = f1_score(x, y_pred,average='macro')
    return f1score

In [3252]:
def F1_cv(model):
    model.fit(X_train,y_train)
    pred = model.predict(X_valid)
    print(F1(y_valid,pred))

In [4263]:
rg_model = RidgeClassifier(random_state=42)
t1 = time.time()
kr_parameters={
    "alpha" : [0.1,1,2,3,4 ,5,6,7], 
    'max_iter':[7,8,9,10,11,12,13],
    'tol':[0.001,0.0001,0.01,0.1,0.2,0.00001],
    'class_weight':['dict','balanced']
    ,'solver':['saga']
    }
    
# RS_iter
n_iter_search =50

# RandomSearchCV
kr_rgs_model = RandomizedSearchCV(
    rg_model, 
    param_distributions=kr_parameters,
    n_iter = n_iter_search,
    cv=3,
    random_state=42,
    n_jobs=-1,
    scoring="f1_macro"
)
kr_rgs_model.fit(train_x, train_y)
t2 = time.time()
print("Processed Time:", str(t2-t1))

Processed Time: 0.2558469772338867


In [4264]:
kr_rgs_model.best_params_

{'tol': 0.01,
 'solver': 'saga',
 'max_iter': 12,
 'class_weight': 'balanced',
 'alpha': 1}

In [4265]:
kr_rgs_model.best_score_

0.9468400801734136

In [4266]:
kr_rgs_best = kr_rgs_model.best_estimator_
pred = kr_rgs_model.predict(test_df)
pred

array(['A', 'B', 'C', 'B', 'A', 'C', 'C', 'B', 'A', 'A', 'C', 'B', 'C',
       'A', 'B', 'B', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'B',
       'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'B', 'B',
       'C', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B',
       'A', 'B', 'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C',
       'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'B',
       'B', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'A', 'B',
       'B', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'C',
       'A', 'A', 'C', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A',
       'C', 'B', 'C', 'A', 'B', 'B', 'C', 'C', 'B', 'B', 'B', 'A', 'B',
       'A', 'A', 'B', 'B', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'A', 'C',
       'B', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A',
       'A', 'B', 'C', 'A', 'B', 'C', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'C', 'C', 'B', 'B'], dtype='<U1')

In [4267]:
lgbm_model = LGBMClassifier(n_jobs =-1, random_state=1234)
lgbm_parameters={
    # 여러 모델을 어떻게 할지
    "n_estimators" : [30,50,60,70 ,100,130],
    "learning_rate" : [0.01,0.005,0.1,0.2,0.3],
    'num_leaves':[20,30,50,70,80,90],
    # 그리고 기본적인 모델 세팅
    "max_depth" : [3,4,5, 7,9],
    "colsample_bytree" : [0.1,0.3,0.4,0.5, 0.6],
  
    }

n_iter_lgbm = 10
lgbm_rgs_model = RandomizedSearchCV(
    lgbm_model,
    param_distributions=lgbm_parameters,
    n_iter = n_iter_lgbm,
    cv= 3,
    scoring= "f1_macro",
    random_state=1234,
    n_jobs=-1,
    verbose=3
)
lgbm_rgs_model.fit(train_x, train_y),

Fitting 3 folds for each of 10 candidates, totalling 30 fits


(RandomizedSearchCV(cv=3, estimator=LGBMClassifier(random_state=1234), n_jobs=-1,
                    param_distributions={'colsample_bytree': [0.1, 0.3, 0.4, 0.5,
                                                              0.6],
                                         'learning_rate': [0.01, 0.005, 0.1, 0.2,
                                                           0.3],
                                         'max_depth': [3, 4, 5, 7, 9],
                                         'n_estimators': [30, 50, 60, 70, 100,
                                                          130],
                                         'num_leaves': [20, 30, 50, 70, 80, 90]},
                    random_state=1234, scoring='f1_macro', verbose=3),)

In [4268]:
lgbm_rgs_model.best_params_

{'num_leaves': 80,
 'n_estimators': 60,
 'max_depth': 7,
 'learning_rate': 0.2,
 'colsample_bytree': 0.1}

In [4269]:
lgbm_rgs_model.best_score_

0.963879694934079

In [4270]:
lgbm_rgs_best = lgbm_rgs_model.best_estimator_
pred = lgbm_rgs_model.predict(test_df)
pred

array(['A', 'B', 'C', 'B', 'A', 'B', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'B', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'B',
       'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'B', 'B',
       'C', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'A', 'B',
       'A', 'B', 'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C',
       'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'B',
       'B', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'A', 'B',
       'B', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'C',
       'A', 'A', 'C', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A',
       'C', 'B', 'B', 'A', 'B', 'B', 'C', 'C', 'B', 'C', 'B', 'A', 'B',
       'A', 'A', 'B', 'B', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'A', 'C',
       'B', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A',
       'A', 'B', 'C', 'A', 'B', 'C', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'C', 'C', 'B', 'B'], dtype=object)

In [2952]:
y = np.array(train_y)

결과확인

In [3966]:
accuracy_score(pred,pddd)

0.9771428571428571

In [2784]:
 #id컬럼 생성
test_df = pd.read_csv('test.csv')
id = test_df.iloc[:,0]
id = pd.DataFrame(id)

In [ ]:
#역변환
pred =lbe_y.inverse_transform(pred)
pred

In [3618]:
list = pd.DataFrame(list)

In [3271]:
pred = pd.DataFrame(pred)

In [3619]:
final = pd.concat([id,list],axis=1)

In [3272]:
final = pd.concat([id,pred],axis=1)

In [3620]:
final = final.rename(columns={0:'class'})

In [3621]:
final = final.dropna()

In [3622]:
final.to_csv('미치겠다deep.csv',index=False)

In [3275]:
final.to_csv('아니화들짝 보팅.csv',index=False)

광기의 딥러닝

In [1494]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
import tensorflow_addons.metrics

In [957]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [2404]:
train_x.shape

(262, 10)

In [3640]:
tf.random.set_seed(42)
dense4 = Dense(units=256, input_dim=22 , activation = 'sigmoid')
dense2 = Dense(units=64, input_dim=256, activation = 'relu')
dense3 = Dense(units=3, input_dim=64, activation = 'softmax')
model = Sequential([dense4,dense2,dense3])
model.compile(loss='categorical_crossentropy',optimizer=Adam(0.003
)
,metrics=[tensorflow_addons.metrics.F1Score(num_classes=3,average='macro')])
model.fit(train_x,train_y,epochs=200)

Epoch 1/200
9/9 [==============================] - 1s 4ms/step - loss: 1.1092 - f1_score: 0.3296
Epoch 2/200
9/9 [==============================] - 0s 3ms/step - loss: 0.8345 - f1_score: 0.6883
Epoch 3/200
9/9 [==============================] - 0s 2ms/step - loss: 0.6799 - f1_score: 0.6563
Epoch 4/200
9/9 [==============================] - 0s 2ms/step - loss: 0.4625 - f1_score: 0.8432
Epoch 5/200
9/9 [==============================] - 0s 3ms/step - loss: 0.3983 - f1_score: 0.8911
Epoch 6/200
9/9 [==============================] - 0s 3ms/step - loss: 0.3171 - f1_score: 0.9057
Epoch 7/200
9/9 [==============================] - 0s 3ms/step - loss: 0.2738 - f1_score: 0.9158
Epoch 8/200
9/9 [==============================] - 0s 4ms/step - loss: 0.2421 - f1_score: 0.9080
Epoch 9/200
9/9 [==============================] - 0s 3ms/step - loss: 0.2374 - f1_score: 0.9208
Epoch 10/200
9/9 [==============================] - 0s 3ms/step - loss: 0.2137 - f1_score: 0.9286
Epoch 11/200
9/9 [===========

In [1971]:
from sklearn.metrics import accuracy_score

In [1972]:
model.save('deep.h5')

In [3641]:
pred = model.predict(test_df)

6/6 [==============================] - 0s 1ms/step


In [3642]:
result = pred.argmax(axis=1)


In [3681]:
list = []
for i in range(len(pred)):
    if result[i] == 0:
        list.append('A')
    elif result[i] == 1:
        list.append('B')
    elif result[i] == 2:
        list.append('C')

In [ ]:
pdd = pred.argmax(axis=1)

In [3166]:
daa = pd.read_csv('탈락빼고원핫3lgbm.csv')

In [3683]:
np.array(list)

array(['A', 'B', 'C', 'B', 'A', 'C', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B',
       'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'B', 'B',
       'C', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B',
       'A', 'B', 'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C',
       'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'B',
       'B', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B',
       'B', 'C', 'B', 'B', 'C', 'A', 'C', 'A', 'C', 'A', 'B', 'B', 'C',
       'A', 'A', 'C', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A',
       'C', 'B', 'B', 'A', 'B', 'B', 'C', 'A', 'B', 'B', 'B', 'A', 'B',
       'A', 'A', 'B', 'B', 'B', 'C', 'A', 'A', 'B', 'B', 'A', 'A', 'C',
       'B', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A',
       'A', 'B', 'C', 'A', 'B', 'C', 'B', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'C', 'C', 'B', 'B'], dtype='<U1')

In [3583]:
pddd = daa['class']

In [3584]:
pddd = np.array(pddd)

In [3617]:
accuracy_score(list,pddd) 

0.9771428571428571

In [4271]:
import sklearn.svm as svm
from scipy import stats
from sklearn.svm import SVC as svc 

mdl = svc(probability = True, random_state = 42)
rand_list = {"C": stats.uniform(2, 20),
             "gamma": stats.uniform(0.1, 1),
             'tol':stats.uniform(0.00001,1),
             
             'degree':stats.uniform(2,10)}
              
rand_search = RandomizedSearchCV(mdl, param_distributions = rand_list, 
n_iter = 20, n_jobs = 4, cv = 3, random_state = 42, scoring ='f1_macro' ) 
rand_search.fit(train_x, train_y) 
rand_search.cv_results_

{'mean_fit_time': array([0.00974162, 0.00591715, 0.00525435, 0.00740806, 0.00933258,
        0.00700037, 0.00702047, 0.01385999, 0.00999848, 0.01134713,
        0.00633502, 0.00846108, 0.01246246, 0.00889754, 0.00562278,
        0.01411215, 0.00543833, 0.01253446, 0.00977397, 0.0093499 ]),
 'std_fit_time': array([0.00117163, 0.00039437, 0.00097884, 0.00121548, 0.00205641,
        0.00081653, 0.00139219, 0.00047339, 0.00185285, 0.00167543,
        0.00186548, 0.00061964, 0.00105112, 0.00083529, 0.00088109,
        0.00043043, 0.0007228 , 0.00171568, 0.00185663, 0.00163963]),
 'mean_score_time': array([0.00506687, 0.00333357, 0.00238196, 0.00299962, 0.00301178,
        0.00270796, 0.00333134, 0.00329137, 0.00356293, 0.00420308,
        0.00294693, 0.00315523, 0.00400376, 0.00521151, 0.00290895,
        0.00337537, 0.00268888, 0.00506282, 0.00323288, 0.00342123]),
 'std_score_time': array([7.53585677e-04, 4.72046670e-04, 9.43903146e-04, 8.16243670e-04,
        8.30297174e-04, 9.75703734e-

In [4272]:
rand_search.best_params_

{'C': 7.175599632000338,
 'degree': 8.62522284353982,
 'gamma': 0.41171107608941093,
 'tol': 0.5200780211778108}

In [4273]:
rand_search.best_score_

0.9569011117956734

In [4274]:
rand_rgs_best = rand_search.best_estimator_
pred = rand_search.predict(test_df)
pred 

array(['A', 'B', 'C', 'B', 'A', 'B', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'B', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'B',
       'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'B', 'B',
       'C', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B',
       'A', 'B', 'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C',
       'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'B',
       'B', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'A', 'B',
       'B', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'C',
       'A', 'A', 'C', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A',
       'C', 'B', 'C', 'A', 'B', 'B', 'C', 'C', 'B', 'C', 'B', 'A', 'B',
       'A', 'A', 'B', 'B', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'A', 'C',
       'B', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A',
       'A', 'B', 'C', 'A', 'B', 'C', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'C', 'C', 'B', 'B'], dtype=object)

In [1314]:
pred2 = model2.predict(test_df)

In [719]:
submit = pd.read_csv('sample_submission.csv')

In [1315]:
submit['class']=pred2

In [1316]:
submit.to_csv('svm_보통.csv',index=False)

In [524]:
from sklearn.cluster import SpectralClustering

In [4275]:
rf = RandomForestClassifier(n_jobs=-1, random_state=1234)
# 1) 파라미터 조합 : RGS -> 좀 범위를 크게크게..
parameters={
    "n_estimators" : [ 100,500,700,750],
    "max_features" :["sqrt","log2"],
    "max_depth" :[2, 3,5, 10,30, 50,60,70],
    "min_samples_split":[6, 10,30,50],
    "min_samples_leaf":[1, 2,3,5, 10]
}
# 2) RGS 몇 번 반복할 것이냐.
n_iter_search = 20
# 3) RGSCV세팅
rf_kf_rgs = RandomizedSearchCV(rf,
                                param_distributions=parameters,
                                cv = 3,
                                scoring="f1_macro",
                                n_jobs = -1 ,
                                random_state= 42,
                                n_iter = n_iter_search
                                )
# 4) 실질적으로 데이터를 밀어서 RGSCV하기위해서 
rf_kf_rgs.fit(train_x, train_y)

RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(n_jobs=-1,
                                                    random_state=1234),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'max_depth': [2, 3, 5, 10, 30, 50, 60,
                                                      70],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 3, 5, 10],
                                        'min_samples_split': [6, 10, 30, 50],
                                        'n_estimators': [100, 500, 700, 750]},
                   random_state=42, scoring='f1_macro')

In [4276]:
rf_kf_rgs.best_score_

0.9599750206260153

In [4277]:
rf_kf_rgs.best_params_

{'n_estimators': 500,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': 60}

In [4148]:
rf_kf_rgs_best = rf_kf_rgs.best_estimator_
pred3 = rf_kf_rgs_best.predict(test_df)
pred3

array(['A', 'B', 'C', 'B', 'A', 'B', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'B', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'B',
       'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'B', 'B',
       'C', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B',
       'A', 'B', 'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C',
       'A', 'C', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'B',
       'B', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'A', 'B',
       'B', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'C',
       'A', 'A', 'C', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A',
       'C', 'B', 'B', 'A', 'B', 'B', 'C', 'C', 'B', 'B', 'B', 'A', 'B',
       'A', 'A', 'B', 'B', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'A', 'C',
       'B', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A',
       'A', 'B', 'C', 'A', 'B', 'C', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'C', 'C', 'B', 'B'], dtype=object)

In [3066]:
submit['class']=pred3

In [1322]:
submit.to_csv('rf_보통.csv',index=False)

In [3067]:
accuracy_score(pred3,pddd)

0.9771428571428571

In [1324]:
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance ## Feature Importance를 불러오기 위함
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [1326]:
params = {'max_depth' : 3,
         'eta' : 0.1, 
         'objective' : 'multi:softmax',
         'eval_metric' : 'logloss',
         'early_stoppings' : 100 }


num_rounds = 400

In [ ]:
np.array(train_x)
np.array(train_y)

In [1334]:
dtrain = xgb.DMatrix(data=train_x, label = train_y)

In [2808]:
from catboost import CatBoostClassifier

In [4278]:
model4 = CatBoostClassifier(iterations=2000,
                           depth=8
                           ,
                           l2_leaf_reg=0.001,
                           
                           learning_rate=0.003,
                           loss_function='MultiClass',
                           verbose=True,random_seed=42,custom_loss=['TotalF1']
                          )
# train the model
model4.fit(train_x, train_y)
# make the prediction using the resulting model
pred4 = model4.predict(test_df)

0:	learn: 1.0934285	total: 1.04ms	remaining: 2.08s
1:	learn: 1.0879694	total: 2.35ms	remaining: 2.35s
2:	learn: 1.0832835	total: 3.21ms	remaining: 2.13s
3:	learn: 1.0794165	total: 4.49ms	remaining: 2.24s
4:	learn: 1.0745326	total: 5.16ms	remaining: 2.06s
5:	learn: 1.0702816	total: 5.6ms	remaining: 1.86s
6:	learn: 1.0654064	total: 6.23ms	remaining: 1.77s
7:	learn: 1.0607073	total: 7.11ms	remaining: 1.77s
8:	learn: 1.0560319	total: 7.98ms	remaining: 1.76s
9:	learn: 1.0516572	total: 8.48ms	remaining: 1.69s
10:	learn: 1.0478183	total: 8.9ms	remaining: 1.61s
11:	learn: 1.0443185	total: 9.28ms	remaining: 1.54s
12:	learn: 1.0397389	total: 10.1ms	remaining: 1.54s
13:	learn: 1.0360887	total: 10.6ms	remaining: 1.51s
14:	learn: 1.0318116	total: 11.5ms	remaining: 1.52s
15:	learn: 1.0282985	total: 12ms	remaining: 1.49s
16:	learn: 1.0240549	total: 12.6ms	remaining: 1.47s
17:	learn: 1.0198518	total: 13.2ms	remaining: 1.45s
18:	learn: 1.0165224	total: 13.5ms	remaining: 1.41s
19:	learn: 1.0127753	total

In [4279]:
pred4

array([['A'],
       ['B'],
       ['C'],
       ['B'],
       ['A'],
       ['B'],
       ['C'],
       ['B'],
       ['A'],
       ['A'],
       ['C'],
       ['B'],
       ['B'],
       ['A'],
       ['B'],
       ['B'],
       ['A'],
       ['B'],
       ['B'],
       ['C'],
       ['B'],
       ['B'],
       ['B'],
       ['A'],
       ['B'],
       ['B'],
       ['B'],
       ['A'],
       ['A'],
       ['B'],
       ['A'],
       ['A'],
       ['B'],
       ['C'],
       ['A'],
       ['B'],
       ['C'],
       ['B'],
       ['B'],
       ['C'],
       ['A'],
       ['B'],
       ['C'],
       ['B'],
       ['B'],
       ['B'],
       ['B'],
       ['C'],
       ['B'],
       ['C'],
       ['A'],
       ['B'],
       ['A'],
       ['B'],
       ['B'],
       ['B'],
       ['C'],
       ['A'],
       ['B'],
       ['C'],
       ['A'],
       ['B'],
       ['C'],
       ['C'],
       ['C'],
       ['A'],
       ['B'],
       ['A'],
       ['A'],
       ['B'],
       ['B'],
      

In [4242]:
accuracy_score(pred4,pddd)

0.0

In [4195]:
from sklearn.ensemble import VotingClassifier

In [4280]:
VC = VotingClassifier(estimators=[('rf',rf),('lgbm',lgbm_rgs_model),('lr',lr_rgs_model)],voting = 'soft')

In [4281]:
VC.fit(train_x,train_y)
pred = VC.predict(test_df)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [4282]:
pred

array(['A', 'B', 'C', 'B', 'A', 'B', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'B', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'B',
       'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'B', 'B',
       'C', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B',
       'A', 'B', 'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C',
       'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'B',
       'B', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'A', 'B',
       'B', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'C',
       'A', 'A', 'C', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A',
       'C', 'B', 'C', 'A', 'B', 'B', 'C', 'C', 'B', 'C', 'B', 'A', 'B',
       'A', 'A', 'B', 'B', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'A', 'C',
       'B', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A',
       'A', 'B', 'C', 'A', 'B', 'C', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'C', 'C', 'B', 'B'], dtype=object)

In [3647]:
from vecstack import StackingTransformer

In [3648]:
estimators=[('rf',rf),('lgbm',lgbm_rgs_model),('lr',lr_rgs_model)]

In [3655]:
from sklearn.metrics import f1_score

In [3967]:
stack = StackingTransformer(estimators, 
                            regression = False, 
                           
                            n_folds = 3, stratified = True, shuffle = True, 
                            random_state = 42) 

In [ ]:
stack = stack.fit(train_x, train_y)
train_x = stack.transform(train_x)
test_df = stack.transform(test_df) 
